In [78]:
from functions import *

In [79]:
params = read_parameters()
subfolder = params['subfolder']
file_idx = params['file_idx']
initial_feed_volume_L = params['initial_feed_volume_L']
reset_columns_when_OFF = params['reset_columns_when_OFF']

parameters:
subfolder: UF/
file_idx_uppaal: 3
file_idx: 0
tmp_idx: 0
initial_feed_volume_L: 100
reset_columns_when_OFF: True
drop_initial_final_off_rows: True
drop_off_rows: True
drop_outliers: True
plot_scatterplot_matrix: False
use_default_arima_params: True
default_arima_params: [1, 1, 0]
include_arima_simulations_in_analysis: True


In [80]:
in_folder = PATH_SENSORS_DATA_RAW_UF
out_folder = PATH_SENSORS_DATA_EXT_UF_V1
cur_file, file_path = get_input_file(in_folder=in_folder, in_file_idx=file_idx, log=True)

file list:
0 /app/data/from_sensors/0_raw/UF/20240711 clean water - reflux valve closed.csv
1 /app/data/from_sensors/0_raw/UF/20240712 clean water - reflux valve open.csv

input file:
/app/data/from_sensors/0_raw/UF/20240711 clean water - reflux valve closed.csv


In [81]:
df_conc = pd.read_excel(FILE_CONCENTRATIONS_DATA)

In [82]:
df = pd.read_csv(file_path)
# print([ type(c) for c in df.columns ])
df.rename(columns=UF_COLUMNS, inplace=True)
df['datetime'] = pd.to_datetime(df["date"] + " " + df["time"], format='%Y/%m/%d %H:%M:%S')
df['time span [s]'] = df['datetime'].diff().dt.total_seconds()
df['time span [s]'] = df['time span [s]'].apply(lambda x : 60 if (58 <= x and x <= 62) else x) # 1-2 secs errors are removed
df['tank liters [L]'] = df['tank liters [%]'] * FEED_TANK_CAPACITY_LITERS / 100
df.loc[0, 'time span [s]'] = 0.0
DATE = df.loc[0, 'datetime'].date().isoformat()
df = df[change_column_index(df.columns.tolist(), 'TMP [bar]', 7)] 
df = df[change_column_index(df.columns.tolist(), 'datetime', 0)] 
df = df[change_column_index(df.columns.tolist(), 'time span [s]', 1)]
df = df[change_column_index(df.columns.tolist(), 'tank liters [L]', 11)]
df = change_unit_measure(df)
df = df.drop(columns=['date', 'time', 'millisecond [ms]'])

In [83]:
#out_file = DATE + re.sub("^[0-9]*", "", cur_file)
out_file = cur_file
out_file_path = out_folder + out_file

In [84]:
# filter df_conc by date
df_conc = df_conc[df_conc['date'] == DATE]

In [85]:
print("area tot included!")
(mbn_model, mbn_num) = MEMBRANE_USED[DATE]
mbn_data = MEMBRANES[mbn_model]
mbn_area = mbn_data['area']
A = mbn_area * mbn_num # m^2
print(f"date: {DATE}, membrane: {mbn_data['producer']}-{mbn_model}, number of membranes: {mbn_num}, total area: {A}")
df['area tot [m^2]'] = A

area tot included!
date: 2024-07-11, membrane: LiqTech-COM0251178, number of membranes: 3, total area: 0.99


In [86]:
print("viscosity included!")
df['prs permeate + ATM [Pa]'] = (df['prs permeate [kPa]'] + PRS_ATM_kpa) * 1000
df['viscosity permeate [Pa s]'] = df.apply((lambda x : calc_viscosity(x['temperature [°C]'], pressure_Pa=x['prs permeate + ATM [Pa]'], element='Water')), axis=1)
df = df[change_column_index(df.columns.tolist(), 'viscosity permeate [Pa s]', 8)]
df = df.drop(columns=['prs permeate + ATM [Pa]'])

viscosity included!


In [87]:
print("flux included!")
# [L / h]   = [m^2]    * [L / m^2 h]
# flow_prmt = area_tot * flux ==> flux = flow_prmt / area_tot
df['flux [L/m^2h]'] = df['flow permeate [L/h]'] / A

flux included!


In [88]:
# print("crossflow velocity included!")

# [m/s]        = [m^3 / h]                 / (  [m^2]  * [s/h] )
# vlct_crsflow = liters__to__m3(flow_feed) / (area_tot * SECS_PER_HRS);
# df['vlct crsflow [m/s]'] = (df['flow feed [L/h]'] / (1000.0 * 3600.0)) / A

# flow_rtnt_m3 = 0.25 * area_tot * (SECS_PER_HRS * vlct_crsflow) * mbn_diam / mbn_len
# [m/s]                    =  [m^3 / h]                            / (  [s / h        ) / (   [m^2]  )
#df['vlct crsflow 2 [m/s]'] = (mbn_len * df['flow retentate [L/h]'] / (1000.0 * 3600.0)) / (0.25 * A * (2 * mbn_radius))

In [89]:
print("pressure input included!")
# PRS_IN = density [kg / m^3] * gravity [m / s^2] * height [m] = 997 * 9.81 * 1.0 = 9780,57 [Pa] = 9,78 [kPa]
df['prs input [kPa]'] = 10.0
df = df[change_column_index(df.columns.tolist(), 'prs input [kPa]', 2)]

pressure input included!


In [90]:
print("total resistance included!")
# res_tot = res_mbn + res_conc_pol + res_fouling
# flux = prs_TMP / (viscosity * res_tot) ==> res_tot = prs_TMP / (viscosity_permeate * flux)
# [1/m]             =     [kPa]       / ([m^3 / m^2 s]                              * [kPa s])
df['res tot [1/m]'] = df['TMP [kPa]'] / ((df['flux [L/m^2h]'] / (1000.0 * 3600.0)) * (df['viscosity permeate [Pa s]'] / 1000.0))

total resistance included!


In [91]:
df['viscosity permeate at 20°C [Pa s]'] = df.apply((lambda x : calc_viscosity(20, pressure_Pa=1000*(PRS_ATM_kpa+x['prs permeate [kPa]']), element='Water')), axis=1)
df['flux at 20° [L/m^2h]'] = df['flux [L/m^2h]'] * df['viscosity permeate [Pa s]'] / df['viscosity permeate at 20°C [Pa s]']
print('permeate viscosity and flux assuming constant temperature of 20°C included!')

permeate viscosity and flux assuming constant temperature of 20°C included!


In [92]:
print("is_ON flag included!")
# add bool flag when machine is ON
df['is_ON'] = df['TMP [kPa]'] >= 5

is_ON flag included!


In [93]:
if reset_columns_when_OFF :
    print(f"setting to zero the following columns when the machine is OFF:")
    for c in df.columns :
        if re.match("^(flow)|(flux)|(res).*", c) :
            print(f" - {c}")
            df[c] = df.apply(reset_cols_if_is_OFF, axis=1, args=('is_ON', c))

setting to zero the following columns when the machine is OFF:
 - flow feed [L/h]
 - flow permeate [L/h]
 - flow retentate [L/h]
 - flux [L/m^2h]
 - res tot [1/m]
 - flux at 20° [L/m^2h]


In [94]:
# add empty columns
for c in ['vol feed [L]', 'vol permeate [L]', 'delta vol permeate [L]', 'time [m]'] :
    print(f"{c} included!")
    df[c] = 0.0
df = df[change_column_index(df.columns.tolist(), 'time [m]', 2)] 
df.loc[0, 'vol feed [L]'] = initial_feed_volume_L
# iterate on rows to calculate values
for i in range(1, len(df)):
    df.loc[i, 'time [m]'] = df.loc[i-1, 'time [m]'] + (df.loc[i, 'time span [s]'] / 60)
    df.loc[i, 'delta vol permeate [L]'] = (df.loc[i-1, 'flow permeate [L/h]'] / 60)
    if (df.loc[i, 'time span [s]'] <= 180) :
        # update
        df.loc[i, 'vol permeate [L]'] = df.loc[i-1, 'vol permeate [L]'] + df.loc[i, 'delta vol permeate [L]']
        df.loc[i, 'vol feed [L]']     = df.loc[i-1, 'vol feed [L]']     - df.loc[i, 'vol permeate [L]']
    else : 
        # reset
        df.loc[i, 'vol permeate [L]'] = 0.0
        df.loc[i, 'vol feed [L]']     = initial_feed_volume_L

vol feed [L] included!
vol permeate [L] included!
delta vol permeate [L] included!
time [m] included!


In [95]:
increased_TMP = []
decreased_TMP = []
starting = []
jumped_temperature = []
if DATE == '2023-11-08' :
    jumped_temperature = [0]
    starting      = [1, 3, 73]
    increased_TMP = [1, 3, 4, 6, 73, 74]
    decreased_TMP = [86, 125, 158, 186, 217]
if DATE == '2023-11-09' :
    jumped_temperature = [5, 6, 32, 153, 154]
    starting      = [154]
    increased_TMP = [6, 37, 42, 48, 56, 87, 90, 91, 93, 95, 154, 155, 161, 165, 168, 171]
    decreased_TMP = [27, 64, 99, 100, 183]
#if DATE == '2023-11-10' :
#    jumped_temperature = [1]
#    starting      = [1]
#    increased_TMP = [1, 2]
#    decreased_TMP = []

In [96]:
df['increased TMP'] = 0
df['decreased TMP'] = 0
df['starting'] = 0
df['jumped temperature'] = 0
for i in increased_TMP :
    df.loc[i, 'increased TMP'] = 1
for i in decreased_TMP :
    df.loc[i, 'decreased TMP'] = 1
for i in starting :
    df.loc[i, 'starting'] = 1
for i in jumped_temperature :
    df.loc[i, 'jumped temperature'] = 1
df['changed TMP'] = ((df['increased TMP'] == 1) | (df['decreased TMP'] == 1) | (df['starting'] == 1))

In [97]:
df.loc[0, 'TMP group'] = 0
for i in range(1, len(df)) :
    cur_group = df.loc[i-1, 'TMP group'] + (0 if df.loc[i, 'changed TMP'] == 0 else 1)
    df.loc[i, 'TMP group'] = cur_group
print('added TMP groups')

added TMP groups


In [98]:
def get_concentration(df_conc, sample_group=0, initial=1, sample_type='feed') :
    df_tmp = df_conc[(df_conc['sample type'] == sample_type) & (df_conc['initial'] == initial) & (df_conc['sample group'] == sample_group)]
    df_tmp = df_tmp.reset_index()
    conc = np.nan
    if not df_tmp.empty :
        conc = df_tmp.loc[0, 'concentration [g/L]']
    return conc

In [99]:
# by default: only one global
conc_interval = CONCENTRATION_INTERVALS.get(out_file, DEFAULT_CONC_GROUP)
for (k,v) in conc_interval.items() :
    print(f"{k}: {v}")

0: ((None, None), 'unknown')


In [100]:
print("splitting df according to different concentrations")
for (group, ((l,r), title)) in conc_interval.items() :
    dfi = df[l:r].reset_index(drop=False)
    dfi['feed concentration [g/L]'] = np.nan
    dfi['retentate concentration [g/L]'] = np.nan
    dfi['permeate concentration [g/L]'] = np.nan
    FIRST = 0
    LAST  = len(dfi) - 1
    dfi.loc[FIRST,      'feed concentration [g/L]'] = get_concentration(df_conc, sample_group=group, initial=1, sample_type='feed')
    dfi.loc[FIRST, 'retentate concentration [g/L]'] = get_concentration(df_conc, sample_group=group, initial=1, sample_type='retentate')
    dfi.loc[FIRST,  'permeate concentration [g/L]'] = get_concentration(df_conc, sample_group=group, initial=1, sample_type='permeate')
    dfi.loc[LAST,  'retentate concentration [g/L]'] = get_concentration(df_conc, sample_group=group, initial=0, sample_type='retentate')
    dfi.loc[LAST,   'permeate concentration [g/L]'] = get_concentration(df_conc, sample_group=group, initial=0, sample_type='permeate')
    # store initial concentration also as a constant column
    dfi['initial feed concentration [g/L]']         = get_concentration(df_conc, sample_group=group, initial=1, sample_type='feed')
    dfi['initial retentate concentration [g/L]']    = get_concentration(df_conc, sample_group=group, initial=1, sample_type='retentate')
    dfi['initial permeate concentration [g/L]']     = get_concentration(df_conc, sample_group=group, initial=1, sample_type='permeate')
    # start from t=1
    dfi['time [m]'] = dfi['time [m]'] - dfi.loc[0, 'time [m]'] + 1
    out_file_i = (out_folder + DATE + ' ' + str(group) + ' ' + title + '.csv')
    # EXPORT according to concentrations
    dfi.to_csv(out_file_i, index=False)

splitting df according to different concentrations


In [101]:
# start from t=1
# df['time [m]'] = df['time [m]'] - df.loc[0, 'time [m]'] + 1

In [102]:
#############################
# EXPORT
# print(f"extended dataset exported as csv: {out_file}")
# df.to_csv(out_file, index=False)